<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Generate Leads from likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_comments_from_post.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #linkedin #post #comments #naas_drivers

## Input

### Import library

In [ ]:
from naas_drivers import linkedin
import pandas as pd
import numpy as np
import naas

### Get your cookies
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [8]:
LI_AT = ''  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = ''  # EXAMPLE ajax:8379907400220387585

### Enter post URL

In [10]:
PROFILE_URL = "PROFILE_URL"

## Model

Get the information return in a dataframe.<br><br>
**Available columns :**
- PROFILE_URN : LinkedIn unique profile id
- PROFILE_ID : LinkedIn public profile id
- FIRSTNAME
- LASTNAME
- TEXT
- OCCUPATION
- ACTIVITY_COMMENTS
- ACTIVITY_LIKES
- DISTANCE
- POST_URL

In [11]:
DF_profile_posts = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_stats(PROFILE_URL)

### Apply necessary filters to initial dataframe

- Get all post from profile
- Filter post with any of the empty value and add to new dataframe


In [12]:
DF_all_post_likes = pd.DataFrame()
for index, row in DF_profile_posts.iterrows():
    df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(row['POST_URL'])
    if not df.empty:
        DF_all_post_likes = DF_all_post_likes.append(df)

- Check the links with <b>PROFILE_ID</b> for users posts. (i.e. Here activity posts are filtered)
- Ignore below line if you want to consider all links)

In [13]:
profile_posts = DF_all_post_likes[DF_all_post_likes['POST_URL'].str.contains('PROFILE_ID')]

In [14]:
grouped_profile_posts = profile_posts.groupby(['PROFILE_ID']).size().sort_values(ascending=False).reset_index(name='count')

- Group like count and classify the leads as per levels i.e. <b>L1, L2, L3, L4.</b> ( from highest to lowest counts of likes) with column name <b>"POTENTIAL LEAD"</b>
- Group posts as per the the column <b>"POTENTIAL LEAD"</b>

In [137]:
conditions = [
    (grouped_profile_posts['count'] <= 2),
    (grouped_profile_posts['count'] > 2) & (grouped_profile_posts['count'] <= 9),
    (grouped_profile_posts['count'] > 9) & (grouped_profile_posts['count'] <= 15),
    (grouped_profile_posts['count'] > 15)
    ]

values = ['L4', 'L3', 'L2', 'L1']

In [138]:
grouped_profile_posts['POTENTIAL_LEAD'] = np.select(conditions, values)

### Get profile details based on the POST_URL above
- Extract EMAIL, FIRSTNAME & LASTNAME from outputs of below cell

In [139]:
PROFILE_CONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(PROFILE_URL)
PROFILE_IDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(PROFILE_URL)

In [140]:
profile_Email = PROFILE_CONTACTS.at[0,'EMAIL']

In [141]:
profile_Name = PROFILE_IDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILE_IDENTITY.at[0,'LASTNAME']

- filter output based on L1, L2, & L3 levels and get profile details of potential leads and add to a list

In [142]:
leads_list = [];

r_count = 1
for index, row in grouped_profile_posts.iterrows():
    if row['POTENTIAL_LEAD'] == "L3" or row['POTENTIAL_LEAD'] == "L2" or row['POTENTIAL_LEAD'] == "L1":
        profileid = row['PROFILE_ID']
        profileurl = "https://www.linkedin.com/in/"+profileid+"/"
        PROFILECONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(profileurl)
        PROFILEIDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(profileurl)
        profileemail = PROFILECONTACTS.at[0,'EMAIL']
        profilename = PROFILEIDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILEIDENTITY.at[0,'LASTNAME']
        if profileemail != None :
            leads_list.append([r_count,profilename,profileemail,profileurl])
            r_count = r_count + 1
   

- Create a dynamic HTML table based on the leads_list generate in above cell

In [143]:
leads_list_str = ""
for i in leads_list:
    leads_list_str = leads_list_str + "<tr><td style='width: 7.04591%;'>{}".format(i[0]) + "</td>" + "<td style='width: 25%;'>"+i[1]+"</td>" +"<td style='width: 25%;'>"+ i[2]+"</td>" +"<td style='width: 25%;'>"+ i[3]+"</td>" +"</tr>"  

## Output

### Display result

- Lead information will be sent in email

In [ ]:
email_to = "EMAIL_ID" #to send the report
subject = "LinkedIN Leads Alert"
content = f"<p>Hi there,</p><br/><p>Here is the list of the leads you generated from your posts on LinkedIN</p><br/><table style='border-collapse: collapse; width: 100%;' border='1'><tbody><tr><td style='width: 3%;'>Sr. No</td><td style='width: 25%;'>Full Name</td><td style='width: 25%;'>Email ID</td><td style='width: 25%;'>LinkedIN Profile</td></tr>" + leads_list_str + "</tbody></table>" + "<br/><p>You can see them in your Hubspot “Contact” section.</p><br/><br/><br/>Best Regards from Naas."

naas.notification.send(email_to=email_to, subject=subject, html=content)